# Create a Gannt chart of Australian government departments

This notebook creates a Gannt-style chart showing the creation and dissolution dates of Australian government departments.

You can [view the Wikidata SPARQL query](https://w.wiki/5tXN) used for this visualisation using the Wikidata Query Service.

In [1]:
import altair as alt
import pandas as pd
from SPARQLWrapper import JSON, SPARQLWrapper

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [3]:
sparql.setQuery(
    """
SELECT
  ?agency ?agencyLabel
  ?naa_id ?start_date ?end_date
WHERE {
  ?agency wdt:P31 wd:Q57605562;
        wdt:P10856 ?naa_id;
        wdt:P571 ?start_date;
  OPTIONAL { ?agency wdt:P576 ?end_date. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} ORDER BY ?start_date
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [4]:
df = pd.json_normalize(results["results"]["bindings"], sep="_")

In [5]:
df.head()

,agency_type,agency_value,naa_id_type,naa_id_value,start_date_datatype,start_date_type,start_date_value,end_date_datatype,end_date_type,end_date_value,agencyLabel_xml:lang,agencyLabel_type,agencyLabel_value
0,uri,http://www.wikidata.org/entity/Q113757341,literal,CA 11,http://www.w3.org/2001/XMLSchema#dateTime,literal,1901-01-01T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1976-12-07T00:00:00Z,en,literal,Department of the Treasury
1,uri,http://www.wikidata.org/entity/Q16959680,literal,CA 6,http://www.w3.org/2001/XMLSchema#dateTime,literal,1901-01-01T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1921-12-21T00:00:00Z,en,literal,Department of Defence
2,uri,http://www.wikidata.org/entity/Q16959770,literal,CA 10,http://www.w3.org/2001/XMLSchema#dateTime,literal,1901-01-01T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1956-01-11T00:00:00Z,en,literal,Department of Trade and Customs
3,uri,http://www.wikidata.org/entity/Q3044789,literal,CA 5,http://www.w3.org/2001/XMLSchema#dateTime,literal,1901-01-01T00:00:00Z,NaN,NaN,NaN,en,literal,Attorney-General's Department
4,uri,http://www.wikidata.org/entity/Q7234360,literal,CA 9,http://www.w3.org/2001/XMLSchema#dateTime,literal,1901-01-01T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1975-12-22T00:00:00Z,en,literal,Postmaster-General's Department


In [6]:
# Convert date fields to date types
df["start_date_value"] = pd.to_datetime(df["start_date_value"])
df["end_date_value"] = pd.to_datetime(df["end_date_value"])
# If there's no end date, set it to now
df[["end_date_value"]] = df[["end_date_value"]].fillna(pd.to_datetime("now", utc=True))

In [7]:
# Create the chart
# The two X values give the start and end of the bar
alt.Chart(df).mark_bar().encode(
    x="start_date_value:T",
    x2="end_date_value:T",
    y=alt.Y("naa_id_value:N", sort="x"),
    color=alt.Color("naa_id_value:N", sort="-x"),
    tooltip=["naa_id_value:N", "agencyLabel_value:N"],
).properties(width=1000)

alt.Chart(...)

----

Created by [Tim Sherratt](https://timsherratt.org) for the [GLAM Workbench](https://glam-workbench.net/).

The development of the Wikidata section of the GLAM Workbench was supported by [Wikimedia Australia](https://wikimedia.org.au/wiki/Wikimedia_Australia).